In [ ]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

# Single-image input inference
image_url = "file:///home/darshan/aks_test/object-detect/image3.png"

chat_response = client.chat.completions.create(
    model="allenai/Molmo-7B-D-0924",
    messages=[{
        "role": "user",
        "content": [
            # NOTE: The prompt formatting with the image token `<image>` is not needed
            # since the prompt will be processed automatically by the API server.
            {"type": "text", "text": "make a bounding box around the fire extinguisher in this image"},
            {"type": "image_url", "image_url": {"url": image_url}},
        ],
    }],
)
print("Chat completion output:", chat_response.choices[0].message.content)


Chat completion output:  <points x1="24.1" y1="74.4" x2="28.0" y2="25.0" x3="76.3" y3="23.6" x4="76.5" y4="74.5" alt="bounding box around the fire extinguisher in this image">bounding box around the fire extinguisher in this image</points>


In [21]:
# display the image with the points
from PIL import Image
import re

image = Image.open('/home/darshan/aks_test/object-detect/image.png')

# get x, y, points from the response
#  <points x1="23.2" y1="24.0" x2="37.4" y2="21.6" x3="52.1" y3="19.1" x4="63.7" y4="19.0" alt="photo frames">photo frames</points>

text = chat_response.choices[0].message.content

# Regular expression to find all x and y coordinate pairs
pattern = r'x\d+="([\d.]+)" y\d+="([\d.]+)"'
coordinates = re.findall(pattern, text)

# Convert to list of tuples of floats
coordinates = [(float(x), float(y)) for x, y in coordinates]
coordinates

[(24.1, 74.4), (28.0, 25.0), (76.3, 23.6), (76.5, 74.5)]

In [ ]:
# display points on the image according to the coordinates
# proper scale is needed to display the points on the image
scale = 1.0
for x, y in coordinates:
    image.putpixel((int(x * scale), int(y * scale)), (255, 0, 0))

image